### ⚠️⚠️⚠️ Este markdown no es necesario volver a ejecutarlo ⚠️⚠️⚠️

### --->> puedes pasar a ejecutar directamente el markdown 4 (asignar labels a cada tile)
 
Las salidas de sete markdown son las siguientes:
- coords/yolo_coords.csv : archivo con las coordenadas en formato YOLO **SIN NORMALIZAR** (hay que usar las normalizadas para entrenar el modelo).
- coords/coords_normalized.csv: archivo con las coordenadas normalizadas perfectas para entrenar el modelo YOLOV8.

In [1]:

import os
import pandas as pd
import rasterio

## Formateo de datos YOLO 
### [class, X_centro, Y_centro, ancho, alto]

Los datos ya están formateados, por lo que esta celda NO HAY QUE EJECUTARLA nunca más.

El objetivo es pasar las coordenadas, que están en X, Y en el csv original , a coordenadas en YOLO, que son:

[class, X_centro, Y_centro, ancho, alto]

In [ ]:

CSV_FILE_ORIGINAL = "G:\\.shortcut-targets-by-id\\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\\doctorado_albert\\conteo_pinguinos\\chinstraps_eca56.csv"
output_path = os.path.join('coords', 'yolo_coords.csv')
IMGS_PATH = os.path.join('tiles_coords', 'subtiles')

In [ ]:

def formatear_datos_yolo(input_csv:str, output_path:str = 'coords/yolo_coords.csv') -> pd.DataFrame:
    """
    Convierte el formato de entrada Id, X, Y, en un formato de salia:
    class,x_center,y_center,width,height
    """
    df = pd.read_csv(input_csv, encoding='ISO-8859-1')
    df_copy = pd.DataFrame()
    df_copy['class'] = df['Id']
    df_copy['x_center'] = df['X']
    df_copy['y_center'] = df['Y']
    df_copy['width'] = 30
    df_copy['height'] = 30
    df_copy.to_csv(output_path, index=False)
    return df

In [ ]:
formatear_datos_yolo(CSV_FILE_ORIGINAL)

## Normalización de datos

El segundo paso para trabajar con yolo es tener los datos normalizados.

In [2]:
FULL_ORTHOMOSAIC = 'G:\\.shortcut-targets-by-id\\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\\doctorado_albert\\conteo_pinguinos\\orthomosaic_all_big.tif'
OUTPUT_PATH_NORMALIZED = 'coords/coords_normalized.csv'
COORDS_YOLO_FORMAT = 'coords/yolo_coords.csv'

In [3]:
def normalize_yolo_coords(
    csv_file_coords:str = COORDS_YOLO_FORMAT, 
    output_file:str = OUTPUT_PATH_NORMALIZED,
    tiff_file:str = FULL_ORTHOMOSAIC
    ) -> None:
    """
    Normaliza todas las coordenadas del dataset completo al formato YOLO.
    
    Args:
    - csv_file: str - Ruta al archivo CSV con todas las coordenadas en formato YOLO (Id, x_center, y_center, width, height).
    - tiff_file: str - Ruta al archivo TIFF que define el área de referencia. Se obtienen así los límites.
    - output_file: str - Nombre del archivo de salida con las coordenadas normalizadas.
    """
    # Cargar datos del CSV
    data = pd.read_csv(csv_file_coords, encoding='ISO-8859-1')

    # Abrir el archivo TIFF para obtener metadatos
    with rasterio.open(tiff_file) as src:
        transform = src.transform
        width, height = src.width, src.height

        # Definir los límites del área total
        top_left = transform * (0, 0)
        bottom_right = transform * (src.width, src.height)
        min_x, max_y = top_left
        max_x, min_y = bottom_right

    # Filtrar los puntos dentro del área total (opcional, por si hay datos fuera del rango del TIFF)
    data = data[
        (data['x_center'] >= min_x) & (data['x_center'] <= max_x) &
        (data['y_center'] >= min_y) & (data['y_center'] <= max_y)
    ]

    # Normalizar las coordenadas y dimensiones
    data['x_center'] = (data['x_center'] - min_x) / (max_x - min_x)  # Normalización en X
    data['y_center'] = (data['y_center'] - min_y) / (max_y - min_y)  # Normalización en Y
    data['width'] = data['width'] / width  # Normalización del ancho
    data['height'] = data['height'] / height  # Normalización de la altura

    # Guardar el dataset en formato YOLO
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    data.to_csv(output_file, index=False, header=False)

    print(f"Dataset normalizado guardado en: {output_file}")

In [4]:

normalize_yolo_coords()

Dataset normalizado guardado en: coords/coords_normalized.csv
